#  Convolutional Neural Network

### Importing the Libraries

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

### Define the image dimensions and batch size

In [10]:
img_width, img_height = 64, 64
batch_size = 32

# Data Preprocessing

### Preprocessing the Training set

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2
                                  )
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 1516 images belonging to 2 classes.


### Preprocessing the Test set

In [12]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 650 images belonging to 2 classes.


# Building the CNN

### Create the CNN model

In [13]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

C:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


# Training the CNN

### Compile the model

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

### Train the model

In [15]:
 model.fit(x = training_set, validation_data = test_set, epochs = 15)


Epoch 1/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.5998 - loss: 0.7470 - val_accuracy: 0.7062 - val_loss: 0.5864
Epoch 2/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.7162 - loss: 0.5679 - val_accuracy: 0.7338 - val_loss: 0.5221
Epoch 3/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.7547 - loss: 0.5262 - val_accuracy: 0.7385 - val_loss: 0.5318
Epoch 4/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.7665 - loss: 0.4957 - val_accuracy: 0.7662 - val_loss: 0.4918
Epoch 5/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 149s 3s/step - accuracy: 0.8048 - loss: 0.4512 - val_accuracy: 0.7569 - val_loss: 0.5097
Epoch 6/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 154s 3s/step - accuracy: 0.7902 - loss: 0.4720 - val_accuracy: 0.7831 - val_loss: 0.5034
Epoch 7/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.8047 - loss: 0.4375 - val_accuracy: 0.7877 - val_loss: 0.5263
Epoch 8/15
48/48 ━━━━━━━━━━━━━━━━━━━━ 147s 3s/step - accuracy: 0.8001 - loss: 0.4351 - val_accuracy: 0.7785 - va

In [17]:
model.save(r'save_model\new_model.keras')

### Evaluate the model

In [18]:
loss, accuracy = model.evaluate(test_set)
print("Test Loss:", loss)
print("Test Accuracy", accuracy)

21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.7804 - loss: 0.5329
Test Loss: 0.5059422254562378
Test Accuracy 0.7953845858573914


### Load the Trained model

In [21]:
model = tf.keras.models.load_model(r'save_model\new_model.keras')

### Set the image path for prediction

In [22]:
image_path = "dataset/single_prediction/malignant.jpg"
print("image_path", image_path)

image_path dataset/single_prediction/malignant.jpg


### Load and preprocess the image

In [23]:
from PIL import Image


def preprocess_image(image_path):
    print(image_path)
    # open the image using IPL
    image = Image.open(image_path)
    print("image", image)
    # Resize the image to the input dimensions expected by the model
    image = image.resize((img_width, img_height), resample=Image.BILINEAR)
    # Convert the array to the Numpy array
    image = np.array(image)
    # Normalize the image
    image = image / 255.0
    # Add an extra dimansion to represent the batch size required by the model
    image = np.expand_dims(image, axis=0)
    return image


### Making a single prediction

In [28]:
# usage
preprocessed_image = preprocess_image(image_path)
# print("preprocessed_image", preprocessed_image)
# make prediction on the preprocesses image
prediction = model.predict(preprocessed_image)
print("prediction=", prediction)

# convert prediction to labels (0 for Benign, 1 for Malignant)
labels = np.round(prediction).flatten()

class_indices = training_set.class_indices

# Determine which class label corresponds to benign and malignant
if class_indices['Benign'] == 0:
    benign_label = 0
    malignant_label = 1
else:
    benign_label = 1
    malignant_label = 0

print("Labels =", labels)
if labels == 0:
    print("Benign")
else:
    print("Malignant")

dataset/single_prediction/malignant.jpg
image <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2854x1348 at 0x20DF8CD6510>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
prediction= [[0.99404365]]
Labels = [1.]
Malignant
